# Código para aplicação das LB direto no dado para rotulação

## Preparando o test da base ouro para aplicar as labels functions

In [2363]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [2364]:
#apenas para testar o iob com spacy
#cria um novo dataset com as 2000 primeiras linhas
#csv_reader = pd.read_csv("/home/lucelia_vieira/Experimentos/ner/CSVs/REL_EXTRATO_CONTRATO_COMPLETO.csv", nrows=200)

In [2365]:
#df = ["EXTRATO DO CONTRATO No 11/2019 PROCESSO: 00390-00003006/2019-92. ESPECIE: Contrato de Aquisicao de Bens pelo Distrito Federal no 11/2019. PARTES: Secretaria de Estado de Desenvolvimento Urbano e Habitacao do Distrito Federal - SEDUH/DF e INDUSTRIA DE AGUA MINERAL IBIA LTDA. DO OBJETO: Aquisicao de agua potavel de mesa sem gas e garrafao retornavel, destinados ao atendimento desta pasta, conforme demais especiGcacoes e condicoes estabelecidas no Termo de Referencia Anexo I do Edital de Pregao Eletronico no 0058/2018 - SCG/SEPLAG, da Proposta da empresa INDUSTRIA DE AGUA MINERAL IBIA LTDA de 11/06/2018 e da Lei no 8.666, de 21 de junho de 1993, que passam a integrar o presente Termo. DO VALOR: R$ 11.661,00 (onze mil, seiscentos e sessenta e um reais) conforme Nota de Empenho no 2019NE00180, emitida em 18/07/2019. Dotacao Orcamentaria P.T 15.122.6001.8517.0131 - Natureza da Despesa: 3390.30 - Fonte de Recursos: 100. DO PRAZO DE VIGENCIA: O Contrato tera vigencia de 06 (seis) meses a contar do dia 07/08/2019. DATA DE ASSINATURA: 22/07/2019. SIGNATARIOS: Pelo Distrito Federal: Mateus Leandro de Oliveira, na qualidade de Secretario de Estado. Pela Contratada: Eduardo Barros de Queiroz Rodrigues, na qualidade de Socio Administrador.", "EXTRATO DO CONTRATO Nº 06/2021 Processo: 00139-00001275/2021-00. Partes: DF - ADMINISTRAÇÃO REGIONAL DO CRUZEIRO X COMERCIAL DE ALIMENTOS PARK SUL. Objeto: O presente contrato tem por objeto a contratação de empresa especializada na área de alimentação para confecção e fornecimento de 50 KG DE BOLO CONFEITADO TIPO ANIVERSÁRIO COM MASSA PÃO DE LÓ COM RECHEIO DE BABA DE MOÇA, BEIJINHO OU DOCE DE LEITE E COBERTURA DE CHANTILLY DECORADO COM ACABAMENTO, no evento de comemoração ao aniversário de 62 anos do Cruzeiro no dia 30/11/2021. Valor do Contrato: R$ 2.000,00 (Dois Mil Reais). Dotação orçamentária: Unidade Orçamentária: 190113. Programa de Trabalho: 13.392.6219.3678.0136 . Natureza da Despesa: 339039. Fonte de Recursos: 120. Nota de Empenho: 2021NE00259. Valor do Empenho Inicial: R$ 2.000,00 (Dois Mil Reais), emitida em 26/11/2021 sob o evento 400091, na Modalidade 1   Ordinário. Fundamento Legal: Art. 24 da Lei nº 8.666/1993. Data da Assinatura: 29/11/2021. Signatários: Pelo DISTRITO FEDERAL: LUIZ EDUARDO GOMES DE PAULA PESSOA, na qualidade de Administrador Regional do Cruzeiro; pela COMERCIAL DE ALIMENTOS PARK SUL: GIVANILDES DE AGUIAR MORAIS, na qualidade de Gerente Geral."] 

In [2366]:
#Carrega o dataset de teste para aplicacao das LB
#data = pd.read_csv('./CSVs/V2/Extrato_Contrato_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Suspensao_Licitacao_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Aditamento_Contrato_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Anul_Revog_Licitacao_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V2/Aviso_Licitacao_Split_0_x_test_texto.csv', dtype=str)
data = pd.read_csv('./CSVs/V2/Extrato_Contrato_Split_0_x_test_texto.csv', dtype=str)


data

,Unnamed: 0,Texto
0,0,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...
1,1,EXTRATO DO CONTRATO DE PRESTAÇÃO DE SERVIÇOS N...
2,2,EXTRATO DO CONTRATO DE AQUISIÇÃO DE PRODUTOS D...
3,3,EXTRATO DO CONTRATO Nº 53/2021 AQUISIÇÃO DE BE...
4,4,EXTRATO DE CONTRATO Processo : 00097-00008203/...
...,...,...
342,342,EXTRATO DO CONTRATO BRB Nº 277/2021 Locador : ...
343,343,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...
344,344,"EXTRATO DO CONTRATO Nº 03/2022 , NOS TERMOS DO..."
345,345,EXTRATO DO CONTRATO Nº 202/2021 PROCESSO : 001...


In [2367]:
#criando novo dataframe somente com os textos
df1 = data['Texto'].reset_index(drop=True)
#df1 = data['texto'].reset_index(drop=True)
df1

0      EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...
1      EXTRATO DO CONTRATO DE PRESTAÇÃO DE SERVIÇOS N...
2      EXTRATO DO CONTRATO DE AQUISIÇÃO DE PRODUTOS D...
3      EXTRATO DO CONTRATO Nº 53/2021 AQUISIÇÃO DE BE...
4      EXTRATO DE CONTRATO Processo : 00097-00008203/...
                             ...                        
342    EXTRATO DO CONTRATO BRB Nº 277/2021 Locador : ...
343    EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...
344    EXTRATO DO CONTRATO Nº 03/2022 , NOS TERMOS DO...
345    EXTRATO DO CONTRATO Nº 202/2021 PROCESSO : 001...
346    EXTRATO DO CONTRATO Nº 71/2021 NOS TERMOS DO P...
Name: Texto, Length: 347, dtype: object

In [2368]:
#len(df_sk)

In [2369]:
# CODIGO DE APLICACAO DA SUPERVISAO FRACA
#Para rodar extrato de contrato descomente abaixo as duas linhas
import dodfSkweak_Contrato
skw = dodfSkweak_Contrato.SkweakContratos(df1)

#Para rodar aditamento de contrato descomente abaixo as duas linhas
#import dodfSkweak_Aditamento
#skw = dodfSkweak_Aditamento.SkweakAditamento(df1)

#Para rodar Aviso Licitação
#import dodfSkweak_Aviso_Licitacao
#skw = dodfSkweak_Aviso_Licitacao.SkweakAvisoLicitacao(df1)

#Para rodar Suspensao descomente abaixo as duas linhas
#import dodfSkweak_Suspensao
#skw = dodfSkweak_Suspensao.SkweakSuspensao(df1)

#Para rodar Anulacao e Revogação
#import dodfSkweak_Anul_Revog
#skw = dodfSkweak_Anul_Revog.SkweakAnulRevog(df1)

#Para rodar Convenio
#import dodfSkweak_Convenio
#skw = dodfSkweak_Convenio.SkweakConvenio(df1)

skw.apply_label_functions()
skw.train_HMM_Dodf()

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


Starting iteration 1
Finished E-step with 347 documents
Starting iteration 2


         1      -19249.3865             +nan


Finished E-step with 347 documents
Starting iteration 3


         2      -18869.7798        +379.6067


Finished E-step with 347 documents
Starting iteration 4


         3      -18848.6035         +21.1764


Finished E-step with 347 documents


         4      -18842.2656          +6.3379


Write to ./data/reuters_small.spacy...done


In [2370]:
skw.list_spans_specific(0)

[15/2021, 00020-00004011/ 2020-46, 10/09/2021, Nota de Empenho nº, 2021NE00245, 120901 - 12901, 03.126.8203.1471.0034, 44.90.40]


In [2371]:
#skw.list_spans_all()

In [2372]:
data_label=skw.get_hmm_dataframe()
data_label

,numero_contrato,processo_gdf,valor_contrato,unidade_orcamentaria,programa_trabalho,natureza_despesa,nota_empenho,data_assinatura_contrato,text,labels,PARTES,CONTRATANTE,CONTRATADA,OBJETO,VIGENCIA
0,15/2021,00020-00004011/ 2020-46,,120901 - 12901,03.126.8203.1471.0034,44.90.40,2021NE00245,Nota de Empenho nº,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,O O O O O O O O B-numero_contrato O O B-proces...,,,,,
1,35/2021,04012-00003310/ 2021-11,"1.215.840,00",25101,11.333.6207.2900.0017,33.90.39,2021NE00659,20/12/2021,EXTRATO DO CONTRATO DE PRESTAÇÃO DE SERVIÇOS N...,O O O O O O O O B-numero_contrato O O O O O B-...,,,,,
2,43/2021,00070-00004971/ 2021-19,"53.328,00(cinquenta",14101,20.306.6201.4115.0007,339032,2021NE00670,17/12/2021,EXTRATO DO CONTRATO DE AQUISIÇÃO DE PRODUTOS D...,O O O O O O O O O O O B-numero_contrato O O B-...,,,,,
3,53/2021,00053-00119399/ 2021-73,"495.843,00",73901,06.181.6217.3029.9512,449052,,05/11/2021,EXTRATO DO CONTRATO Nº 53/2021 AQUISIÇÃO DE BE...,O O O O B-numero_contrato O O O O O B-processo...,,,,,
4,,00097-00008203/ 2020-83,"474.998,40",,26.453.6216.2756.6137,33.90.30,2021NE00580,21/06/2021,EXTRATO DE CONTRATO Processo : 00097-00008203/...,O O O O O B-processo_gdf I-processo_gdf O O O ...,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,277/2021,041.001.563/2021,"1.620.000,00",,,,,,EXTRATO DO CONTRATO BRB Nº 277/2021 Locador : ...,O O O O O B-numero_contrato O O O O O O O O O ...,,,,,
343,10/2021,00146-00000624/ 2021-13,"617,27",9118,15.451.6209.8508.0010,,2021NE00163,04/10/2021,EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...,O O O O O O O O B-numero_contrato O O O O O O ...,,,,,
344,03/2022,00052-00014928/ 2021-71,"63.175,00",73901,28.845.0903.00NR.0053,339040 - 23,2022NE000203,10/02/2022,"EXTRATO DO CONTRATO Nº 03/2022 , NOS TERMOS DO...",O O O O B-numero_contrato O O O O O O O O O O ...,,,,,
345,202/2021,00150-00000604/ 2021-66,"200,00",,13.392.8219.4220.0011 -,33.90.35,00461/2021,,EXTRATO DO CONTRATO Nº 202/2021 PROCESSO : 001...,O O O O B-numero_contrato O O B-processo_gdf I...,,,,,


In [2373]:
# converting to CSV file
#data_label.to_csv("./CSVs/ANUL_REVOG_SUPERVISÃO_FRACA_TESTE.csv")

In [2374]:
corpus = skw.get_IOB()

In [2375]:
len(corpus)

347

In [2376]:
termos_skweak = []
debug=set()
labels_skweak = []
for ato in corpus:
    x = []
    y = []
    for termo in ato:
        termo_label = termo.split()
        x.append(termo_label[0])
        y.append(termo_label[1])
        debug.add(termo_label[1])
    termos_skweak.append(x)
    labels_skweak.append(y)


In [2377]:
debug

{'B-data_assinatura_contrato',
 'B-natureza_despesa',
 'B-nota_empenho',
 'B-numero_contrato',
 'B-processo_gdf',
 'B-programa_trabalho',
 'B-unidade_orcamentaria',
 'B-valor_contrato',
 'I-data_assinatura_contrato',
 'I-natureza_despesa',
 'I-nota_empenho',
 'I-numero_contrato',
 'I-processo_gdf',
 'I-programa_trabalho',
 'I-unidade_orcamentaria',
 'O'}

In [2378]:
len(termos_skweak)

347

In [2379]:
termos_skweak_list = []
for i in termos_skweak:
   if not isinstance(i, list):
      termos_skweak_list .append(i)
   else:
      for j in i:
       termos_skweak_list .append(j)

len(termos_skweak_list )

85679

In [2380]:
#termos_skweak_list

In [2381]:
#apenas para conferencia
#df = pd.DataFrame(termos_skweak_list, columns=['Texto']) 
#df.to_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/termos_skweak_list.csv')

In [2382]:
labels_skweak_list = []
for i in labels_skweak:
   if not isinstance(i, list):
      labels_skweak_list.append(i)
   else:
      for j in i:
        labels_skweak_list.append(j)

len(labels_skweak_list)

85679

## Preparando o test da base ouro para fazer a validação

In [2383]:
#Carrega o dataset de teste da base ouro para aplicar a validação
#df_gold_ato= pd.read_csv('./CSVs/Extrato_Contrato_Split_1_x_test.csv', dtype=str)
#df_gold_ato

In [2384]:
#Carrega os atos do dataset de teste da base ouro
#https://pt.stackoverflow.com/questions/232208/ler-uma-lista-de-listas-em-txt-python-3
import pickle
#Contrato
#with open("./CSVs/V2/Extrato_Contrato_Split_0_x_test.txt", "rb") as file:
#Aditamento
#with open("./CSVs/V2/Aditamento_Contrato_Split_0_x_test.txt", "rb") as file:
#Suspensao
#with open("./CSVs/V2/Suspensao_Licitacao_Split_0_x_test.txt", "rb") as file:
#Aviso Licitacao
with open("./CSVs/V2/Extrato_Contrato_Split_0_x_test.txt", "rb") as file: 
#Anul e Revog
#with open("./CSVs/V2/Suspensao_Licitação_Split_4_x_test.txt", "rb") as file: 
#Convenio
#with open("./CSVs/V2/Extrato_Convenio_Split_4_x_test.txt", "rb") as file:

      
    x_test = pickle.load(file)

In [2385]:
# Retira uma nível da lista dos atos do dataset de teste da base ouro
x_test_list = []
for i in x_test:
   if not isinstance(i, list):
      x_test_list.append(i)
   else:
      for j in i:
        x_test_list.append(j)

len(x_test_list)

85680

In [2386]:
#Carrega os labels do dataset de teste da base ouro (Lista dentro de lista)
import pickle
#Contrato
#with open("./CSVs/V2/Extrato_Contrato_Split_0_y_test.txt", "rb") as file:
#Aditamento
#with open("./CSVs/V2/Aditamento_Contrato_Split_0_y_test.txt", "rb") as file:
#Suspensao
#with open("./CSVs/V2/Suspensao_Licitacao_Split_0_y_test.txt", "rb") as file:
#Aviso Licitacao
with open("./CSVs/V2/Extrato_Contrato_Split_0_y_test.txt", "rb") as file: 
#Anul e Revog
#with open("./CSVs/V2/Suspensao_Licitação_Split_0_y_test.txt", "rb") as file:
#Convenio
#with open("./CSVs/V2/Extrato_Convenio_Split_0_y_test.txt", "rb") as file:        
    y_test = pickle.load(file)

In [2387]:
#  Retira uma nível da lista dos labels do dataset de teste da base ouro

y_test_list = []
for i in y_test:
   if not isinstance(i, list):
      y_test_list.append(i)
   else:
      for j in i:
       y_test_list.append(j)

len(y_test_list)

85680

In [2388]:
len(termos_skweak_list)

85679

In [2389]:
#Apenas checa se os tokens estão batento
resultados = []
for sk_ato , gold_ato in zip(termos_skweak_list, x_test_list):
    tupla=(sk_ato , gold_ato) 
    resultados.append(tupla)
    if sk_ato != gold_ato:
        print(sk_ato, gold_ato)

#for item in resultados:
#    print(item)


, ,firmado
firmado entre
entre a
a SECRETARIA
SECRETARIA DE
DE ESTADO
ESTADO DE
DE PROTECAO
PROTECAO DA
DA ORDEM
ORDEM URBANISTICA
URBANISTICA DO
DO DISTRITO
DISTRITO FEDERAL-DF
FEDERAL-DF LEGAL
LEGAL x
x RR
RR CLIMATIZACAO
CLIMATIZACAO LTDA
LTDA ME
ME :
: O
O presente
presente instrumento
instrumento ,
, tem
tem por
por objeto
objeto aquisicao
aquisicao de
de aparelhos
aparelhos de
de ar
ar condicionado
condicionado ,
, com
com instalacao
instalacao simples
simples ,
, tipo
tipo Split
Split com
com Sistema
Sistema Inverter
Inverter ,
, itens
itens 01,03
01,03 e
e 06
06 ;
; PE
PE n
n 0122/2018
0122/2018 ;
; Unidade
Unidade Orcamentaria
Orcamentaria :
: 63101
63101 ;
; Programa
Programa de
de Trabalho
Trabalho :
: 04.122.6003.8517.0125;Natureza
04.122.6003.8517.0125;Natureza da
da Despesa
Despesa :
: 44.90.52
44.90.52 ;
; Fonte
Fonte de
de Recursos
Recursos :
: 100
100 ;
; no
no valor
valor de
de R$
R$ 108.365,00
108.365,00 (
( cento
cento e
e oito
oito mil
mil trezentos
trezentos e
e s

In [2390]:

''' classes = ['B-numero_contrato',
 'B-processo_gdf',
 'I-processo_gdf',
 'B-data_assinatura_contrato',
 'I-numero_contrato',
 'I-data_assinatura_contrato'] '''


" classes = ['B-numero_contrato',\n 'B-processo_gdf',\n 'I-processo_gdf',\n 'B-data_assinatura_contrato',\n 'I-numero_contrato',\n 'I-data_assinatura_contrato'] "

In [2391]:
#len(labels_skweak_list)

In [2392]:
#classes

In [2393]:
#Validação com sklearn
#metrics.flat_f1_score(y_test, labels_skweak, average='weighted', labels=classes)

In [2394]:
#Validação com seqeval
from seqeval.metrics import f1_score
f1_score(y_test, labels_skweak, average='weighted')

ValueError: Found input variables with inconsistent numbers of samples:
[224, 303, 291, 140, 144, 342, 168, 146, 147, 192, 185, 263, 192, 425, 51, 253, 244, 340, 292, 311, 242, 345, 291, 80, 227, 272, 258, 286, 279, 336, 311, 193, 191, 246, 315, 419, 116, 406, 206, 248, 178, 311, 557, 114, 209, 160, 432, 120, 321, 190, 165, 231, 124, 819, 150, 109, 191, 170, 198, 188, 240, 151, 190, 242, 412, 343, 226, 251, 142, 318, 266, 189, 179, 189, 336, 156, 82, 381, 333, 564, 259, 253, 253, 286, 155, 355, 381, 192, 253, 141, 242, 226, 317, 175, 160, 120, 154, 409, 226, 241, 175, 718, 119, 296, 77, 164, 171, 254, 306, 291, 339, 325, 162, 285, 231, 272, 258, 241, 214, 298, 256, 259, 271, 159, 186, 111, 247, 127, 191, 186, 259, 112, 112, 285, 157, 426, 239, 135, 396, 255, 257, 259, 187, 110, 338, 342, 405, 246, 269, 238, 171, 217, 173, 345, 113, 116, 248, 301, 323, 113, 346, 231, 196, 192, 315, 228, 253, 272, 303, 310, 505, 193, 168, 222, 167, 254, 300, 160, 207, 148, 303, 232, 236, 187, 265, 241, 2

In [ ]:
# classification_report Seqeval
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

report = classification_report(y_test, labels_skweak, output_dict=False, mode='strict', scheme=IOB2)
print(report)


                              precision    recall  f1-score   support

                          11       0.00      0.00      0.00         1
                          12       0.00      0.00      0.00         1
                codigo_siggo       0.00      0.00      0.00        25
                data_escrito       0.80      0.74      0.77       293
            nome_responsavel       0.00      0.00      0.00        18
             numero_contrato       0.92      0.73      0.82       381
        numero_termo_aditivo       0.00      0.00      0.00       329
objeto_aditamento_contratual       0.00      0.00      0.00       301
           orgao_contratante       0.00      0.00      0.00       357
                processo_gdf       0.82      0.97      0.89       287

                   micro avg       0.85      0.39      0.53      1993
                   macro avg       0.26      0.24      0.25      1993
                weighted avg       0.41      0.39      0.40      1993



/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# classification_report Seqeval
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

report = classification_report(y_test, labels_skweak, output_dict=True, mode='strict', scheme=IOB2)
print(report)
np.save("./Results/V2/METRICA_EXTRATO_CONTRATO_LB_DIRETO_0.npy", report)

{'11': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, '12': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'codigo_siggo': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25}, 'data_escrito': {'precision': 0.8029739776951673, 'recall': 0.7372013651877133, 'f1-score': 0.7686832740213524, 'support': 293}, 'nome_responsavel': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 18}, 'numero_contrato': {'precision': 0.9240924092409241, 'recall': 0.7349081364829396, 'f1-score': 0.8187134502923976, 'support': 381}, 'numero_termo_aditivo': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 329}, 'objeto_aditamento_contratual': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 301}, 'orgao_contratante': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 357}, 'processo_gdf': {'precision': 0.8249258160237388, 'recall': 0.9686411149825784, 'f1-score': 0.891025641025641, 'support': 287}, 'micro avg': {'

In [ ]:
#classification_report sklearn
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
   gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=False)
print(report)

NameError: name 'gold_y_test' is not defined

In [ ]:
import numpy as np
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=True)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_LB_DIRETO_V1_0.npy", report)